In [85]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<img width=50px  src = 'https://apps.fs.usda.gov/lcms-viewer/images/lcms-icon.png'>

# Lab 8: Export and Post-processing

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/redcastle-resources/lcms-training/blob/main/8-Export_and_Post_Processing.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/redcastle-resources/lcms-training/blob/main/8-Export_and_Post_Processing.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://github.com/redcastle-resources/lcms-training/blob/main/8-Export_and_Post_Processing.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>
<br/><br/><br/>


## 8.0: Overview and Introduction


This notebook teaches how to export data from GEE into a final platform-agnostic format


### 8.0.1: Objective

In this tutorial, you learn how to export the final thematic LCMS outputs to local CogTiffs, set custom no data values, update the names, and color map.

This tutorial uses the following Google Cloud services:

- `Google Earth Engine`

The steps performed include:

- Exporting assets to Google Cloud Storage
- Copying from Google Cloud Storage to a local directory
- Mosaicking and updating output color map and names
- Ensuring final output is a working CogTiff

### 8.0.2: Before you begin

#### If you are working in Workbench: Set your current URL under `workbench_url`
This gives the Map Viewer a url in which to host the viewer we will be generating. 
* This will be in your URL/search bar at the top of the browser window you are currently in
* It will look something like `https://1234567890122-dot-us-west3.notebooks.googleusercontent.com/` (See the image below)

![workspace url](img/workspace-url.png)

#### Set a folder to use for all exports under `export_path_root` 
* This folder should be an assets folder in an existing GEE project.
* By default, this folder is the same as the pre-baked folder (where outputs have already been created). 
* If you would like to create your own outputs, specify a different path for `export_path_root`, but leave the `pre_baked_path_root` as it was. This way, the pre-baked outputs can be shown at the end, instead of waiting for all exports to finish.
* It will be something like `projects/projectID/assets/newFolder`
* This folder does not have to already exist. If it does not exist, it will be created

**If you are working in Qwiklabs and wish to export:** Copy the project ID from the 'Start Lab' screen into the `projectID` field in `export_path_root`.

In [30]:
workbench_url = 'https://1307eb830a12e633-dot-us-central1.notebooks.googleusercontent.com/'
pre_baked_path_root  = 'projects/rcr-gee/assets/lcms-training'
export_path_root = pre_baked_path_root

print('Done')

Done


#### Installation
First, install necessary Python packages. Uncomment the first line to upgrade geeViz if necessary.

Note that for this module, we're also importing many data science packages such as pandas. 

In [38]:
#Module imports
#!python -m pip install geeViz --upgrade
try:
    import geeViz.getImagesLib as getImagesLib
except:
    !python -m pip install geeViz
    import geeViz.getImagesLib as getImagesLib

import geeViz.changeDetectionLib as changeDetectionLib
import geeViz.assetManagerLib as aml
import geeViz.taskManagerLib as tml

import os,secrets


ee = getImagesLib.ee
Map = getImagesLib.Map

# Can set the port used for viewing map outputs
Map.port = 1235


# Set up GDAL
try:
    import lcms_scripts.raster_processing_lib as rpl
except Exception as e:
    if getImagesLib.IS_WORKBENCH:
        print('GDAL is not installed. Will try to install')
        !sudo apt-get -y install libgdal-dev
        !gdal-config --version
        !pip install pygdal==3.2.2.10
        import lcms_scripts.raster_processing_lib as rpl
    else:
        print('GDAL is not installed and you are not using Workbench.')
    
print('Done')

Done


#### Set up your work environment

Create a folder in your export path where you will export the composites. In addition, create a blank image collection where your composites will live.

Currently, when running within Colab or Workbench, geeView uses a different project to authenticate through, so you may need to make your asset public to view from within Colab.

In [43]:
# Bring in all folders/collections that are needed
# These must already exist as they are created in previous notebooks
export_assembledLCMSOutputs_collection = f'{export_path_root}/lcms-training_module-6_assembledLCMSOutputs'

token = secrets.token_hex(8)
export_bucket = f'gs://lcms-outputs-temp2'
!gsutil mb {export_bucket}


local_processing_folder = '/tmp/lcms-training/local_processing'
if not os.path.exists(local_processing_folder):os.makedirs(local_processing_folder)

print('Done')

Creating gs://lcms-outputs-temp2/...
ServiceException: 409 A Cloud Storage bucket named 'lcms-outputs-temp2' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.
Done


In [36]:
# set up map
Map.clearMap()

# reset port if necessary
Map.port = 1235
Map.proxy_url = workbench_url

print('Done')

Done


## 8.1: Bring in Assembled LCMS Outputs and visualize

In [57]:
Map.clearMap()
# Bring in assembled LCMS asset
lcms = ee.ImageCollection(export_assembledLCMSOutputs_collection)

# Add the assembled outputs to the map and take a look
Map.addLayer(lcms,{},'Assembled LCMS')

Map.turnOnInspector()
Map.view()


Adding layer: Assembled LCMS
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Local web server at: http://localhost:1235/geeView/ already serving.
cwd /home/jupyter/lcms-training
Workbench Proxy URL: https://1307eb830a12e633-dot-us-central1.notebooks.googleusercontent.com/proxy/1235/geeView/?accessToken=None


127.0.0.1 - - [18/Sep/2023 20:08:25] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -


## 8.2: Export Assembled LCMS Outputs to Google Cloud Storage

In [29]:
Map.clearMap()

# Pull the projection info from the assembled asset
projection = lcms.first().projection().getInfo()
if 'crs' in projection.keys():
    crs = projection['crs']
else:
    crs = projection['wkt']
transform = projection['transform']
scale=None


# Get studyArea 
study_area = lcms.first().geometry().bounds(500,crs)

# Find the years
years = [int(float(yr)) for yr in lcms.aggregate_histogram('year').keys().getInfo()]

# Find the products
products = lcms.first().bandNames().slice(0,3).getInfo()

# Set null value
nodata_value = 0

# Get the study area name
study_area_name = lcms.first().get('study_area').getInfo()

# Set version number
version = 'vTraining2022-8'

# Iterate across each product and year and export to cloud storage
for product in products:
    lcms_class = lcms.select([product])
    
    for year in years:
        lcms_class_yr = lcms_class.filter(ee.Filter.calendarRange(year,year,'year')).first()
        
        output_name = f'LCMS_{study_area_name}_{version}_{product}_Annual_{year}'
        
        # getImagesLib.exportToCloudStorageWrapper(lcms_class_yr,output_name,os.path.basename(export_bucket),study_area,scale,crs,transform,outputNoData = nodata_value)


NameError: name 'lcms' is not defined

In [26]:
# Can track tasks here or at https://code.earthengine.google.com/tasks
# If you'd like to track the tasks, use this:
tml.trackTasks2()

# If you want to cancel all running tasks, you can use this function
# tml.batchCancel()

# If you want to empty the bucket of all images
# !gsutil rm {export_bucket}*.tif

print('done')

0 tasks ready 2023-09-19 16:05:30
0 tasks running 2023-09-19 16:05:30
Running names:


done


## 8.3: Copy to local folder
* Look at what is in the bucket
* Copy to local folder for post-processing

In [6]:
# First open this link to view the outputs
print(f'https://console.cloud.google.com/storage/browser/{os.path.basename(export_bucket)}')

https://console.cloud.google.com/storage/browser/lcms-outputs-temp2


In [27]:
# Copy to local folder and make sure it worked
!gsutil -m cp {export_bucket}/*.tif {local_processing_folder}

print(os.listdir(local_processing_folder))

Copying gs://lcms-outputs-temp2/LCMS_PRUSVI_vTraining2022-8_Change_Annual_1985.tif...
Copying gs://lcms-outputs-temp2/LCMS_PRUSVI_vTraining2022-8_Change_Annual_1988.tif...
Copying gs://lcms-outputs-temp2/LCMS_PRUSVI_vTraining2022-8_Change_Annual_1987.tif...
Copying gs://lcms-outputs-temp2/LCMS_PRUSVI_vTraining2022-8_Change_Annual_1986.tif...
Copying gs://lcms-outputs-temp2/LCMS_PRUSVI_vTraining2022-8_Change_Annual_1989.tif...
Copying gs://lcms-outputs-temp2/LCMS_PRUSVI_vTraining2022-8_Change_Annual_1990.tif...
Copying gs://lcms-outputs-temp2/LCMS_PRUSVI_vTraining2022-8_Change_Annual_2001.tif...
Copying gs://lcms-outputs-temp2/LCMS_PRUSVI_vTraining2022-8_Change_Annual_1991.tif...
Copying gs://lcms-outputs-temp2/LCMS_PRUSVI_vTraining2022-8_Change_Annual_1992.tif...
Copying gs://lcms-outputs-temp2/LCMS_PRUSVI_vTraining2022-8_Change_Annual_1993.tif...
Copying gs://lcms-outputs-temp2/LCMS_PRUSVI_vTraining2022-8_Change_Annual_2000.tif...
Copying gs://lcms-outputs-temp2/LCMS_PRUSVI_vTraining2

## 8.4 Create VRTs

* Sometimes if the raster is very large, GEE will break an individual image into tiled outputs
* We will need to ensure those get mosaicked back together 
* We will look for sets of images with the same name
* Note that for the default test study area of Puerto Rico and the US Virgin Islands, the outputs are not broken into tiles

In [28]:
# Find sets of images
for product in products:
    for year in years:
        print(product,year)


NameError: name 'products' is not defined

### 6.3.1: Set up Random Forest Model for thresholding

In [25]:
# Split the training data as we did before into dependent (y) and independent (X) variables
X = training_df[predictor_variable_names]
y = training_df[code_field_name]
groups = training_df['PLOTID']
# Split them into a test and train set for further model evaluation
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Ensure all plots are grouped into either test or split so temporally auto-correlated 
# plots do not get into the training set and inflate accuracy
gss = GroupShuffleSplit(n_splits=1, train_size=0.8)
for i, (train_index, test_index) in enumerate(gss.split(X, y, groups)):
    train_index = train_index
    test_index = test_index

X_train, X_test, y_train, y_test = X.iloc[train_index],X.iloc[test_index],y.iloc[train_index],y.iloc[test_index]

# Set up a random forest model
print('Fitting models for thresholding')
rf_all = RandomForestClassifier(n_estimators = nTrees, random_state=seed,oob_score=True,)
rf_holdout = RandomForestClassifier(n_estimators = nTrees, random_state=seed,oob_score=True,)

# Fit using all training as well as the holdout
rf_all = rf_all.fit(X,y)
rf_holdout = rf_holdout.fit(X_train,y_train)

# Apply the model and get the proportion of votes for each class
rf_all_probs = rf_all.predict_proba(X)
rf_holdout_probs = rf_holdout.predict_proba(X_test)


print('Done')

Fitting models for thresholding
Done


### 6.3.2: Compute proportion of votes that balance omission and comission
* Next we'll compute the proportion of votes that balances omission and commission


* While this assemblage method works well, you can introduce various rules to tailor the final map product to different audiences
* In the case of LCMS, there is a focus on forest applications
* We know that our models are generally not very confident about classifying change largely due to the limited number of training samples that experience change
* We also know our land use classes are not mutually exclusive (e.g. non-forest wetlands can potentially occur in any land use)
* Therefore we have a lot of confusion in our land use outputs

* Since a pixel may have probabilities above the threshold of more than one class, we choose the class with the model confidence that is the highest probability that is also above its respective threshold. 
* e.g. if the fast loss probability is 0.5 and the slow loss probability is 0.35, the pixel will be assigned fast loss
* We will not include stable since stable will be any pixel that is not already assigned to one of the 3 change classes

In [13]:
# Next we'll compute the proportion of votes that balances omission and commission

# Specify individual class names and codes for change
change_class_codes = {'Stable':1,'Slow Loss':2,'Fast Loss':3,'Gain':4}

# Make a function that will take a list of binary reference and probablity (0-1 float) 
# predicted values and find the optimal threshold to balance omission and commission
def getBalancedThreshold(y_ref_binary,y_pred_probs,title='Omission-Commission Error',thresholds = list(np.arange(0,1,0.005))):
    
    omission = []
    commission = []
    
    # Iterate across each threshold and compute the omission and commision error rates
    for threshold in thresholds:
        
        y_pred_binary = np.where(y_pred_probs>threshold, 1, 0)
        fp = (y_pred_binary == 1) & (y_ref_binary == 0) 
        pred_positive = y_pred_binary == 1
        ref_positive = y_ref_binary == 1
        ref_negative = y_ref_binary == 0
        fn = (y_pred_binary == 0) & (y_ref_binary == 1) 
        tp = (y_pred_binary == 1) & (y_ref_binary == 1) 
        commission.append(sum(fp)/sum(pred_positive))
        omission.append(sum(fn)/sum(ref_positive))
    
    # Find the threshold that corresponds to the smallest difference between omission and commission
    diff = np.abs(np.array(omission)-np.array(commission))
    diff = np.stack((thresholds,diff,omission,commission),1)
    diff = diff[diff[:, 1].argsort()]
    crossover = diff[0]
    optimal_thresh = crossover[0]
    
    # Plot the curves
    plt.plot(thresholds, commission,label='Commission')
    plt.plot(thresholds,omission, '-.',label='Omission')
    plt.axvline(x = optimal_thresh, color = 'g', label = f'Optimal Threshold ({optimal_thresh} {crossover[2]} {crossover[3]})')
    plt.xlabel("Proportion RF Model Votes")
    plt.ylabel("Error Rate")
    plt.title(title)
    plt.legend()
    plt.show()
    
    
    return optimal_thresh

# Iterate across each class and find the optimal threshold
out_threshold_dict = {}
for change_class,change_code in list(change_class_codes.items()):
    print(change_class,change_code)
    y_pred_prob_rf_all =rf_all_probs[:,change_code-1]
    y_pred_prob_rf_holdout =rf_holdout_probs[:,change_code-1]
    
    y_ref_binary = np.where(y==change_code, 1, 0)
    y_test_binary = np.where(y_test==change_code,1,0)
    rf_all_thresh = getBalancedThreshold(y_ref_binary,y_pred_prob_rf_all,f'All Training Plots - {change_class}')
    rf_holdout_thresh = getBalancedThreshold(y_test_binary,y_pred_prob_rf_holdout,f'Held Out Training Plots - {change_class}')
    out_threshold_dict[change_class] = rf_holdout_thresh

print(out_threshold_dict)

# Save table for use 
model_options_csv_filename = os.path.join(local_model_data_folder,'LCMS_change_thresholds.csv')
o = open(model_options_csv_filename,'w')
o.write(json.dumps(out_threshold_dict))
o.close()

o = open(model_options_csv_filename,'r')
threshs = json.load(o)
o.close()


print('Done')

NameError: name 'np' is not defined

* Notice if we don't use a holdout, the threshold always hovers near 0.5
* This is wrong since the model contains the data being used to derive the threshold
* Also notice the slow loss threshold is near zero with an error rate near 1. Recall we have very few slow loss samples. This indicates our models are unlikely to be able to predict slow loss at all.
* Note that both fast loss and gain have thresholds far below 0.5 and error rates around 0.7 or so. This indicates our models are rarely confident of classifying these classes, and the omission and commission error rates are high. Simply put, TimeSync is much more sensitive to detecting change than any model is.

## 6.3.3: Final Change Layer Assemblage

In [10]:

Map.clearMap()
# For assembling change, we choose thresholds that balance omission and commission
# These were computed in module 5.1
change_thresholds = { 'Slow Loss': 0.29, 'Fast Loss': 0.28, 'Gain': 0.36}



product = 'Change'

# Pull the class names
class_names = lcms_viz_dict[f'{product}_class_names']
class_codes = lcms_viz_dict[f'{product}_class_values']
class_dict = dict(zip(class_names,class_codes))

# Pull the codes in the correct order
class_order_codes = [class_dict[cl] for cl in list(change_thresholds.keys())]

# Filter to only include the product of interest
raw_lcms_product = raw_lcms.filter(ee.Filter.eq('product',product))
    
# Function to assemble change for a given year
# Uses a specified set of bands and thresholds in order to find the highest probability class above its threshold
# It then fills in the stable value as any remaining non null value
# Nulls are then recoded to the non processing code
def assembleChange(img,product):
    # Convert array to multi-band image
    img = ee.Image(arrayToImage(img,lcms_viz_dict[f'{product}_class_names'][:-1]))
    
    # Select only the bands we'd like to assemble
    img = img.select(list(change_thresholds.keys()))
    
    # Find the most confident class
    maxConf = img.reduce(ee.Reducer.max())
    
    # Find pixels that are the most confident and above the threshold
    maxConfMask = img.eq(maxConf).And(img.gte(list(change_thresholds.values()))).selfMask()
    
    # Get class code for any pixel that is above its threshold for the most confident class
    maxClass = ee.Image(class_order_codes).updateMask(maxConfMask).reduce(ee.Reducer.first())
    
    # Fill stable back in
    maxClass = maxClass.unmask(1)
    
    # Burn in non processing where the input was null
    maxClass = maxClass.where(img.mask().reduce(ee.Reducer.min()).Not(),class_codes[-1])
    
    return maxClass.rename([product]).copyProperties(img).copyProperties(img,['system:time_start'])

# Iterate across the change probabilities and assemble final change class
assembledChange = raw_lcms_product.map(lambda img:assembleChange(img,product).set(lcms_viz_dict))

# Explore it on the map
Map.addTimeLapse(assembledChange,{'autoViz':True},'Assembled LCMS Change')

Map.turnOnInspector()
Map.view()

Adding layer: Assembled LCMS Change
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Local web server at: http://localhost:1235/geeView/ already serving.
cwd /home/jupyter/lcms-training
Workbench Proxy URL: https://7c39a91cfd2f6ff9-dot-us-central1.notebooks.googleusercontent.com/proxy/1235/geeView/?accessToken=None


127.0.0.1 - - [12/Sep/2023 22:56:26] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -


#### Inspect

In [11]:
Map.clearMap()
# Viewing change outputs can be easier as a composite that shows the most recent year of a single change type
# This example will show the most recent year of fast loss

# First, add a year constant band
assembledChange = assembledChange.map(getImagesLib.addYearBand)

# Then mask where change equals fast loss (3)
fastLoss = assembledChange.map(lambda img: img.updateMask(img.select([0]).eq(3)))

# Find the most recent year
mostRecentFastLoss = fastLoss.max().select(['year'])
Map.addLayer(mostRecentFastLoss,{'min':1985,'max':2022,'palette':changeDetectionLib.lossYearPalette},'Most Recent Year of Fast Loss')

Map.turnOnInspector()
Map.view()
print(assembledChange.first().bandNames().getInfo())

Adding layer: Most Recent Year of Fast Loss
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Local web server at: http://localhost:1235/geeView/ already serving.
cwd /home/jupyter/lcms-training
Workbench Proxy URL: https://7c39a91cfd2f6ff9-dot-us-central1.notebooks.googleusercontent.com/proxy/1235/geeView/?accessToken=None


['Change', 'year']


127.0.0.1 - - [12/Sep/2023 23:10:44] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -


## 6.4: Export Assembled assets

* Now lets export assembled assets

* OPTIONAL!!!!
* Optionally, we can export using the tile grid approach
* For PRUSVI, LCMS does not need to export using this approach, but this is how you would set it up
* First, we'll set up the study area and a tile to export across

In [12]:
Map.clearMap()
# Now lets export assembled assets

# OPTIONAL!!!!
# Optionally, we can export using the tile grid approach
# For PRUSVI, LCMS does not need to export using this approach, but this is how you would set it up
# First, we'll set up the study area and a tile to export across

studyArea = ee.FeatureCollection('projects/lcms-292214/assets/R8/PR_USVI/Ancillary/prusvi_boundary')

# Set the size (in meters) of the tiles
# We can likely use a large tile for this step
# If exports fail, reducing the tileSize is likely to help
tileSize = 240000


# Set the projection
crs = getImagesLib.common_projections['NLCD_CONUS']['crs']
transform  = getImagesLib.common_projections['NLCD_CONUS']['transform']
scale = None
projection = ee.Projection(crs,transform)


# Get the grid
grid = studyArea.geometry().coveringGrid(projection.atScale(tileSize))
Map.addLayer(grid,{},'Tile Grid {}m'.format(tileSize))

Map.centerObject(grid)
Map.view()

Adding layer: Tile Grid 240000m
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Local web server at: http://localhost:1235/geeView/ already serving.
cwd /home/jupyter/lcms-training
Workbench Proxy URL: https://7c39a91cfd2f6ff9-dot-us-central1.notebooks.googleusercontent.com/proxy/1235/geeView/?accessToken=None


127.0.0.1 - - [12/Sep/2023 23:10:58] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -


* Iterate across each year and assemble full stack output
* [Example of fully assembed bands](https://developers.google.com/earth-engine/datasets/catalog/USFS_GTAC_LCMS_v2022-8#bands)
* We will not be producing the qa-bits bands

In [41]:
# Iterate across each year, assemble full stack output 
# products = ['Land_Cover','Land_Use']
Map.clearMap()

# Provide a study area name and version
study_area_name = 'PRUSVI'
version = '2022-Training'

# Set up years to apply models across
apply_years = list(range(1985,2022+1))


# This will be populated on the first iteration
pyramidingPolicy = None


for apply_year in apply_years:
    assembled_list = []
    probability_list = []
    for product in products:
        # Pull the class names
        class_names = lcms_viz_dict[f'{product}_class_names'][:-1]
        
        # Get rid of missing class names
        class_names = [nm for nm in class_names if nm not in missing_names[product]]
        
        # Get rid of any characters that aren't allowed
        class_names = [nm.replace(' ','-').replace('/','-').replace('&','and') for nm in class_names]
        
        probability_names = [f'{product}_Raw_Probability_{class_name}' for class_name in class_names]
        
        product_title = product.replace('_',' ')

        # Filter to only include the product of interest
        raw_lcms_product_yr = raw_lcms\
                            .filter(ee.Filter.calendarRange(apply_year,apply_year,'year'))\
                            .filter(ee.Filter.eq('product',product)).first()
        
        if product in ['Land_Cover','Land_Use']:
            # Find the max probability class
            maxProb_lcms_product_yr = getMostProbableClass(raw_lcms_product_yr,product)
            assembled_list.append(maxProb_lcms_product_yr)
        
        elif product == 'Change':
            change_lcms_product_yr = assembleChange(raw_lcms_product_yr,product)
            assembled_list.append(change_lcms_product_yr)
            
        # Convert raw model probabilities to bands for exploration
        prob_lcms_product_yr = arrayToImage(raw_lcms_product_yr,probability_names)
        probability_list.append(prob_lcms_product_yr)
        
    final_stack = ee.Image(ee.Image.cat([ee.Image.cat(assembled_list).byte(),ee.Image.cat(probability_list).multiply(100)]).byte()\
                    .copyProperties(raw_lcms_product_yr,['study_area','year']))

    exportName = f'LCMS_{study_area_name}_v{version}_{apply_year}'
    exportPath = f'{export_assembledLCMSOutputs_collection}/{exportName}'

    # Set up a pyramiding object for each band
    # Final output bands are thematic and need a mode resampling method
    # Probability bands are continuous, so mean is appropriate
    if pyramidingPolicy == None:
        bns = final_stack.bandNames().getInfo()
        pyramidingPolicy = {}
        for bn in bns:
            if bn.find('_Probability_') == -1:
                pyramidingPolicy[bn] = 'mode'
            else:
                pyramidingPolicy[bn] = 'mean'
   
    getImagesLib.exportToAssetWrapper(final_stack,exportName,exportPath,pyramidingPolicy,studyArea,scale,crs,transform,overwrite=True)

print('Done')

Exporting: LCMS_PRUSVI_v2022-Training_1985
Exporting: LCMS_PRUSVI_v2022-Training_1986
Exporting: LCMS_PRUSVI_v2022-Training_1987
Exporting: LCMS_PRUSVI_v2022-Training_1988
Exporting: LCMS_PRUSVI_v2022-Training_1989
Exporting: LCMS_PRUSVI_v2022-Training_1990
Exporting: LCMS_PRUSVI_v2022-Training_1991
Exporting: LCMS_PRUSVI_v2022-Training_1992
Exporting: LCMS_PRUSVI_v2022-Training_1993
Exporting: LCMS_PRUSVI_v2022-Training_1994
Exporting: LCMS_PRUSVI_v2022-Training_1995
Exporting: LCMS_PRUSVI_v2022-Training_1996
Exporting: LCMS_PRUSVI_v2022-Training_1997
Exporting: LCMS_PRUSVI_v2022-Training_1998
Exporting: LCMS_PRUSVI_v2022-Training_1999
Exporting: LCMS_PRUSVI_v2022-Training_2000
Exporting: LCMS_PRUSVI_v2022-Training_2001
Exporting: LCMS_PRUSVI_v2022-Training_2002
Exporting: LCMS_PRUSVI_v2022-Training_2003
Exporting: LCMS_PRUSVI_v2022-Training_2004
Exporting: LCMS_PRUSVI_v2022-Training_2005
Exporting: LCMS_PRUSVI_v2022-Training_2006
Exporting: LCMS_PRUSVI_v2022-Training_2007
Exporting: 

In [ ]:
# Can track tasks here or at https://code.earthengine.google.com/tasks
# If you'd like to track the tasks, use this:
tml.trackTasks2()

# If you want to cancel all running tasks, you can use this function
# tml.batchCancel()

# If you want to empty the collection of all images
# aml.batchDelete(export_rawLCMSOutputs_collection, type = 'imageCollection')

print('done')

21 tasks ready 2023-08-22 02:57:57
4 tasks running 2023-08-22 02:57:57
Running names:
['LCMS_PRUSVI_v2022-Training_2001', '0:00:44']
['LCMS_PRUSVI_v2022-Training_2000', '0:05:39']
['LCMS_PRUSVI_v2022-Training_1999', '0:09:12']
['LCMS_PRUSVI_v2022-Training_1997', '0:12:41']


21 tasks ready 2023-08-22 02:58:02
4 tasks running 2023-08-22 02:58:02
Running names:
['LCMS_PRUSVI_v2022-Training_2001', '0:00:49']
['LCMS_PRUSVI_v2022-Training_2000', '0:05:44']
['LCMS_PRUSVI_v2022-Training_1999', '0:09:17']
['LCMS_PRUSVI_v2022-Training_1997', '0:12:47']


Serving HTTP on 0.0.0.0 port 4321 (http://0.0.0.0:4321/) ...

Keyboard interrupt received, exiting.


### Inspect final outputs

In [10]:
# View final outputs
Map.clearMap()
lcms = ee.ImageCollection(export_assembledLCMSOutputs_collection)

Map.addLayer(lcms,{'opacity':0},'Full Stack Output')
lcms = lcms.map(lambda img:img.set(lcms_viz_dict))
for product in products:Map.addTimeLapse(lcms.select([product]),{'autoViz':True},f'LCMS {product}')

Map.turnOnInspector()
Map.view()

Adding layer: Full Stack Output
Adding layer: LCMS Change
Adding layer: LCMS Land_Cover
Adding layer: LCMS Land_Use
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Local web server at: http://localhost:1235/geeView/ already serving.
cwd /home/jupyter/lcms-training
Workbench Proxy URL: https://53c21733d8125e22-dot-us-west3.notebooks.googleusercontent.com/proxy/1235/geeView/?accessToken=None


127.0.0.1 - - [22/Aug/2023 03:48:56] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -


## Lab 6 challenge:
TBD

In [ ]:
# insert challenge code here

## Done with Module 6